In [46]:
# 02_log_creation
# Create the log from the CSV of TED 

In [47]:
# timestamp in the CSV:
# DT_DISPATCH
# CONTRACT_START
# CONTRACT_COMPLETION
# DT_APPLICATIONS
# DT_AWARD

In [48]:
import pandas as pd
import os
import csv

In [ ]:
# FUNCTIONS

In [187]:
def log_entry(case_id, event, timestamp, type_con, amount, electronic, framework, nuts, cpv_main, country_code):
    """ Given the ID, the event name, its timestamp and static attributes, create an event in the log file """
    str_csv = str(case_id) + ";" + str(event) + ";" + str(timestamp) + ";" + str(type_con) + ";" + str(amount) + ";" + str(electronic) + ";" + str(framework) + ";" + str(nuts) + ";" + str(cpv_main) + ";" + str(country_code) + os.linesep
    with open(path_log, "a") as fp:
        fp.write(str_csv)

In [188]:
def fix_date(date):
    """ Given a date with named months, transforms it into a date with numeric months """
    dic = {'JAN': '01', 'FEB': '02', 'MAR':'03', 'APR':'04', 'MAY':'05', 'JUN': '06', 
           'JUL':'07', 'AUG':'08', 'SEP':'09', 'OCT':'10', 'NOV':'11', 'DEC':'12'}
    parts = date.split("-")
    dd = str(parts[0])
    mm = str(parts[1])
    yy = str(parts[2])
    
    new_yy = "20" + parts[2]
    new_dd = dd
    new_mm = dic[str(mm)]
    
    new_date = new_yy+"-"+new_mm+"-"+new_dd
    
    return new_date

In [249]:
# GLOBALS
country_code = "IT" # <-- INPUT: desired country [IT, FR, ES]
cae_type = "3" # <-- INPUT: CAE_TYPE -> 3 = "Regional or local authority"
log_dir = "log" # <-- log dir to save the files

file_name = "Tender_join_Award_2016_2022_"+country_code+"_Type_"+cae_type+"_clean.csv"
log_file = "log_ted_awd_2016-2023_"+country_code+"_type_"+cae_type+".csv"

# amount threshold
ts_w = 5382000 # works
ts_su = 140000 # supplies and services

In [ ]:
### MAIN ###

In [250]:
# Reading the CSV ted
dt = {'ID_NOTICE_CN':object, 'MAIN_CPV_CODE_GPA':object}
df_ted_awd = pd.read_csv(file_name, sep = ";", dtype=dt, low_memory=False)

In [251]:
df_ted_awd.columns

Index(['ID_NOTICE_CN', 'TED_NOTICE_URL', 'YEAR', 'ID_TYPE', 'DT_DISPATCH',
       'XSD_VERSION', 'CANCELLED', 'CORRECTIONS', 'FUTURE_CAN_ID',
       'FUTURE_CAN_ID_ESTIMATED', 'B_MULTIPLE_CAE', 'CAE_NAME',
       'CAE_NATIONALID', 'CAE_ADDRESS', 'CAE_TOWN', 'CAE_POSTAL_CODE',
       'ISO_COUNTRY_CODE', 'B_MULTIPLE_COUNTRY', 'ISO_COUNTRY_CODE_ALL',
       'CAE_TYPE', 'EU_INST_CODE', 'MAIN_ACTIVITY', 'B_ON_BEHALF',
       'B_INVOLVES_JOINT_PROCUREMENT', 'B_AWARDED_BY_CENTRAL_BODY',
       'TYPE_OF_CONTRACT', 'TAL_LOCATION_NUTS', 'B_FRA_AGREEMENT',
       'FRA_ESTIMATED', 'B_FRA_SINGLE_OPERATOR', 'FRA_NUMBER_OPERATORS',
       'FRA_NUMBER_MAX_OPERATORS', 'B_DYN_PURCH_SYST', 'CPV', 'ID_LOT',
       'ADDITIONAL_CPVS', 'B_GPA', 'LOTS_NUMBER', 'LOTS_SUBMISSION',
       'B_VARIANTS', 'VALUE_EURO', 'VALUE_EURO_FIN_1', 'VALUE_EURO_FIN_2',
       'B_OPTIONS', 'B_EU_FUNDS', 'B_RENEWALS', 'DURATION', 'CONTRACT_START',
       'CONTRACT_COMPLETION', 'TOP_TYPE', 'B_ACCELERATED', 'ENV_OPERATORS',
     

In [252]:
df_ted_awd.shape

(42841, 96)

In [253]:
df_ted_awd.columns

Index(['ID_NOTICE_CN', 'TED_NOTICE_URL', 'YEAR', 'ID_TYPE', 'DT_DISPATCH',
       'XSD_VERSION', 'CANCELLED', 'CORRECTIONS', 'FUTURE_CAN_ID',
       'FUTURE_CAN_ID_ESTIMATED', 'B_MULTIPLE_CAE', 'CAE_NAME',
       'CAE_NATIONALID', 'CAE_ADDRESS', 'CAE_TOWN', 'CAE_POSTAL_CODE',
       'ISO_COUNTRY_CODE', 'B_MULTIPLE_COUNTRY', 'ISO_COUNTRY_CODE_ALL',
       'CAE_TYPE', 'EU_INST_CODE', 'MAIN_ACTIVITY', 'B_ON_BEHALF',
       'B_INVOLVES_JOINT_PROCUREMENT', 'B_AWARDED_BY_CENTRAL_BODY',
       'TYPE_OF_CONTRACT', 'TAL_LOCATION_NUTS', 'B_FRA_AGREEMENT',
       'FRA_ESTIMATED', 'B_FRA_SINGLE_OPERATOR', 'FRA_NUMBER_OPERATORS',
       'FRA_NUMBER_MAX_OPERATORS', 'B_DYN_PURCH_SYST', 'CPV', 'ID_LOT',
       'ADDITIONAL_CPVS', 'B_GPA', 'LOTS_NUMBER', 'LOTS_SUBMISSION',
       'B_VARIANTS', 'VALUE_EURO', 'VALUE_EURO_FIN_1', 'VALUE_EURO_FIN_2',
       'B_OPTIONS', 'B_EU_FUNDS', 'B_RENEWALS', 'DURATION', 'CONTRACT_START',
       'CONTRACT_COMPLETION', 'TOP_TYPE', 'B_ACCELERATED', 'ENV_OPERATORS',
     

In [254]:
df_ted_awd.head()

,ID_NOTICE_CN,TED_NOTICE_URL,YEAR,ID_TYPE,DT_DISPATCH,XSD_VERSION,CANCELLED,CORRECTIONS,FUTURE_CAN_ID,FUTURE_CAN_ID_ESTIMATED,...,NUMBER_TENDERS_OTHER_EU,NUMBER_TENDERS_SME,OUT_OF_DIRECTIVES,TITLE,WIN_ADDRESS,WIN_COUNTRY_CODE,WIN_NAME,WIN_NATIONALID,WIN_POSTAL_CODE,WIN_TOWN
0,201632,ted.europa.eu/udl?uri=TED:NOTICE:32-2016:TEXT:...,2016,2,30-DEC-15,R209.S1,0,0,2016225139,0,...,NaN,NaN,0,Adquisición de 8 vehículos todo terreno tipo p...,Poligono del Tambre; Vía Isaac Peral 18,ES,Caeiro Rey; S.L.,B70384169,15890,Santiago de Compostela
1,201632,ted.europa.eu/udl?uri=TED:NOTICE:32-2016:TEXT:...,2016,2,30-DEC-15,R209.S1,0,0,2016225139,0,...,NaN,NaN,0,Adquisición de 16 remolques para la atención a...,c/ Casimiro Gómez; 21 bj,ES,Peycar Pontevedra; S.L.,B36112043,36002,Pontevedra
2,201632,ted.europa.eu/udl?uri=TED:NOTICE:32-2016:TEXT:...,2016,2,30-DEC-15,R209.S1,0,0,2016225139,0,...,NaN,NaN,0,Adquisición de 3 embarcaciones neumáticas de s...,Camiño Serra da Auga; 26,ES,Narwhal Boats; S.L.,B54743935,36416,Mos
3,201632,ted.europa.eu/udl?uri=TED:NOTICE:32-2016:TEXT:...,2016,2,30-DEC-15,R209.S1,0,0,2016225139,0,...,NaN,NaN,0,Adquisición de 16 remolques de salvamento y re...,Casimiro Gómez; 21 bj,ES,Peycar Pontevedra; S.L.,B36112043,36002,Pontevedra
4,201636,ted.europa.eu/udl?uri=TED:NOTICE:36-2016:TEXT:...,2016,2,30-DEC-15,R209.S1,0,0,2016208156,0,...,NaN,4.0,0,Servicio de desarrollo de una campaña para pro...,Daimler; 38; Polígono Espíritu Santo; Cambre,ES,Imprenta Mundo; S.L.U.,B15389513,15660,La Coruña


In [255]:
# get intest columns
list_col = ['ID_NOTICE_CN', 'DT_DISPATCH', 'CONTRACT_START', 'CONTRACT_COMPLETION', 
            'DT_APPLICATIONS', 'DT_AWARD', 'TYPE_OF_CONTRACT', 'VALUE_EURO',
           'B_ELECTRONIC_AUCTION', 'B_FRA_AGREEMENT', 'TAL_LOCATION_NUTS', 'MAIN_CPV_CODE_GPA']
df_log = df_ted_awd[list_col]

In [256]:
df_log.head()

,ID_NOTICE_CN,DT_DISPATCH,CONTRACT_START,CONTRACT_COMPLETION,DT_APPLICATIONS,DT_AWARD,TYPE_OF_CONTRACT,VALUE_EURO,B_ELECTRONIC_AUCTION,B_FRA_AGREEMENT,TAL_LOCATION_NUTS,MAIN_CPV_CODE_GPA
0,201632,30-DEC-15,NaN,NaN,08-FEB-16,13-MAY-16,U,689028.00,N,N,ES11,34
1,201632,30-DEC-15,NaN,NaN,08-FEB-16,22-JUN-16,U,689028.00,N,N,ES11,34
2,201632,30-DEC-15,NaN,NaN,08-FEB-16,25-MAY-16,U,689028.00,N,N,ES11,34
3,201632,30-DEC-15,NaN,NaN,08-FEB-16,10-MAY-16,U,689028.00,N,N,ES11,34
4,201636,30-DEC-15,NaN,NaN,08-FEB-16,13-MAY-16,S,669834.71,N,N,ES11,793


In [257]:
# df_log['TAL_LOCATION_NUTS'].unique()

In [258]:
df_log.columns

Index(['ID_NOTICE_CN', 'DT_DISPATCH', 'CONTRACT_START', 'CONTRACT_COMPLETION',
       'DT_APPLICATIONS', 'DT_AWARD', 'TYPE_OF_CONTRACT', 'VALUE_EURO',
       'B_ELECTRONIC_AUCTION', 'B_FRA_AGREEMENT', 'TAL_LOCATION_NUTS',
       'MAIN_CPV_CODE_GPA'],
      dtype='object')

In [ ]:
# Create the log from the CSV

In [259]:
path_log = os.path.join(log_dir, log_file)

with open(path_log, "w") as fp:
    fp.write("case_id;event;timestamp;type;amount;electronic;framework_agr;nuts;cpv_main;country"+os.linesep)
    
for row in df_log.itertuples():
    
    if pd.isna(row[1]):
        continue
    
    # case id
    case_id = str(row[1])
    
    # static attributes
    
    # tender type
    if pd.isna(row[7]):
        type_con = ""
    else:
        type_con = str(row[7])
        
    # tender value
    if pd.isna(row[8]):
        amount = ""
    else:
        amount = str(row[8])
        
    # electronic auction
    if pd.isna(row[9]):
        electronic = ""
    else:
        electronic = str(row[9])
        
    # framework agreement
    if pd.isna(row[10]):
        framework = ""
    else:
        framework = str(row[10])
        
    # PA nuts
    if pd.isna(row[11]):
        nuts = ""
    else:
        nuts = str(row[11]).replace(";","").replace("ITDITD","ITD")
        
    # main CPV code
    if pd.isna(row[12]):
        cpv_main = ""
    else:
        cpv_main = str(row[12]).replace(";","")
    
    # events of the case id
    if pd.isna(row[2]):
         dispatch = ""
    else:
        dispatch = str(row[2])
        log_entry(case_id, "PUBLICATION", fix_date(dispatch), type_con, amount, electronic, framework, nuts, cpv_main, country_code)
    
    if pd.isna(row[5]):
        deadline = ""
    else:
        deadline = str(row[5])
        log_entry(case_id, "PARTICIPATION", fix_date(deadline), type_con, amount, electronic, framework, nuts, cpv_main, country_code)
    
    if pd.isna(row[6]):
        award = ""
    else:
        award = str(row[6])
        log_entry(case_id, "AWARD", fix_date(award), type_con, amount, electronic, framework, nuts, cpv_main, country_code)
    
    if pd.isna(row[3]):
        contract_start = ""
    else:
        contract_start = str(row[3])
        log_entry(case_id, "CONTRACT-START", fix_date(contract_start), type_con, amount, electronic, framework, nuts, cpv_main, country_code)
    
    if pd.isna(row[4]):
        contract_end = ""
    else:
        contract_end = str(row[4])
        log_entry(case_id, "CONTRACT-END", fix_date(contract_end), type_con, amount, electronic, framework, nuts, cpv_main, country_code)